---
title: Assignment 03
author:
  - name: Bhargavi Manyala
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
  pdf: 
    fig-format: png
    embed-resources: true
    toc-depth: 2
    geometry: 
      - landscape
      - margin=0.5in
    
date-modified: today
date-format: long
jupyter: assignment03-venv
execute:
  echo: true
  eval: true
  output: true
  freeze: auto
---


# Import Dataset


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from pyspark.sql import SparkSession
import re
import numpy as np
import plotly.graph_objects as go
from pyspark.sql.functions import col, split, explode, regexp_replace, transform, when
from pyspark.sql import functions as F
from pyspark.sql.functions import col, monotonically_increasing_id

np.random.seed(42)
pio.renderers.default = "notebook+png"
 
spark = SparkSession.builder.appName("LightcastData").getOrCreate()
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("escape", "\"")  
    .csv("lightcast_job_postings.csv")
)

#df.show(5)


# Relational Tables 

## Loactions Table
`LOCATION_ID (Primary Key)`,`LOCATION`, `CITY_NAME`, `STATE_NAME`, `COUNTY_NAME`,`MSA`, `MSA_NAME`

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id


locations_df = (
    df.select(
        "LOCATION",
        "CITY_NAME",
        "STATE_NAME",
        "COUNTY_NAME",
        "MSA",
        "MSA_NAME"
    )
    .distinct()
    .withColumn("LOCATION_ID", monotonically_increasing_id())
    .select(
        "LOCATION_ID",
        "LOCATION",
        "CITY_NAME",
        "STATE_NAME",
        "COUNTY_NAME",
        "MSA",
        "MSA_NAME"
    )
    .orderBy("LOCATION_ID")
)

locations_df.createOrReplaceTempView("locations")

locations = locations_df.toPandas()
locations.head()

## Companies Table
`COMPANY_ID (Primary Key)`, `COMPANY`, `COMPANY_NAME`, `COMPANY_RAW`, `COMPANY_IS_STAFFING`

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

companies_df = (
    df.select(
        "COMPANY",
        "COMPANY_NAME",
        "COMPANY_RAW",
        "COMPANY_IS_STAFFING"
    )
    .distinct()
    .withColumn("COMPANY_ID", monotonically_increasing_id())
    .select(
        "COMPANY_ID",
        "COMPANY",
        "COMPANY_NAME",
        "COMPANY_RAW",
        "COMPANY_IS_STAFFING"
    )
    .orderBy("COMPANY_ID")
)

companies_df.createOrReplaceTempView("companies")

companies = companies_df.toPandas()
companies.head(4)

## Industries Table

`INDUSTRY_ID (Primary Key)`, `NAICS_2022_6`, `NAICS_2022_6_NAME`, `SOC_5`, `SOC_5_NAME`,`LOT_SPECIALIZED_OCCUPATION_NAME`,`LOT_OCCUPATION_GROUP`

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

industries_df = (
    df.select(
        "NAICS_2022_6",
        "NAICS_2022_6_NAME",
        "SOC_5",
        "SOC_5_NAME",
        "LOT_SPECIALIZED_OCCUPATION_NAME",
        "LOT_OCCUPATION_GROUP"
    )
    .distinct()
    .withColumn("INDUSTRY_ID", monotonically_increasing_id())
    .select(
        "INDUSTRY_ID",
        "NAICS_2022_6",
        "NAICS_2022_6_NAME",
        "SOC_5",
        "SOC_5_NAME",
        "LOT_SPECIALIZED_OCCUPATION_NAME",
        "LOT_OCCUPATION_GROUP"
    )
    .orderBy("INDUSTRY_ID")
)

industries_df.createOrReplaceTempView("industries")

industries = industries_df.toPandas()
industries.head()

## Job Postings Table

`ID (Primary Key)`, `TITLE_CLEAN`, `COMPANY_ID (FK to companies)`, `INDUSTRY_ID (FKto industries)`, `EMPLOYMENT_TYPE_NAME`, `REMOTE_TYPE_NAME`, `BODY`,`MIN_YEARS_EXPERIENCE`, `MAX_YEARS_EXPERIENCE`, `SALARY`, `SALARY_FROM`,`SALARY_TO`, `LOCATION_ID (FK to locations)`, `POSTED`, `EXPIRED`, `DURATION`


In [ ]:
job_postings_df = df.select(
    "ID",
    "TITLE_CLEAN",
    "BODY",
    "COMPANY",            
    "EMPLOYMENT_TYPE_NAME",
    "REMOTE_TYPE_NAME",
    "MIN_YEARS_EXPERIENCE",
    "MAX_YEARS_EXPERIENCE",
    "SALARY",
    "SALARY_FROM",
    "SALARY_TO",
    "LOCATION",           
    "NAICS_2022_6",       
    "POSTED",
    "EXPIRED",
    "DURATION"
).withColumnRenamed("ID", "JOB_ID").orderBy("JOB_ID")


### Adding Foreign Keys to Job Postings Table

In [ ]:
job_postings_df = job_postings_df.join(
    companies_df.select("COMPANY", "COMPANY_ID"),
    on="COMPANY", 
    how="left"
)

# Join with Locations Table to get LOCATION_ID
job_postings_df = job_postings_df.join(
    locations_df.select("LOCATION", "LOCATION_ID"),
    job_postings_df.LOCATION == locations_df.LOCATION,
    how="left"
).drop("LOCATION")

# Join with Industries Table to get INDUSTRY_ID
job_postings_df = job_postings_df.join(
    industries_df.select("NAICS_2022_6", "INDUSTRY_ID"),
    job_postings_df.NAICS_2022_6 == industries_df.NAICS_2022_6,
    how="left"
).drop("NAICS_2022_6")

job_postings_df = job_postings_df.drop("COMPANY", "LAT-LONG")
job_postings_df.createOrReplaceTempView("job_postings")

job_postings_df.show(truncate=10)

#  Casting salary and experience columns

## Computing medians and Imputing missing salaries

In [ ]:
from pyspark.sql.functions import col

df = df.withColumn("SALARY", col("SALARY").cast("float")) \
       .withColumn("SALARY_FROM", col("SALARY_FROM").cast("float")) \
       .withColumn("SALARY_TO", col("SALARY_TO").cast("float")) \
       .withColumn("MIN_YEARS_EXPERIENCE", col("MIN_YEARS_EXPERIENCE").cast("float")) \
       .withColumn("MAX_YEARS_EXPERIENCE", col("MAX_YEARS_EXPERIENCE").cast("float"))

# Computing medians for salary columns
def compute_median(sdf, col_name):
    return sdf.approxQuantile(col_name, [0.5], 0.01)[0]

median_from = compute_median(df, "SALARY_FROM")
median_to   = compute_median(df, "SALARY_TO")
median_salary = compute_median(df, "SALARY")

print("Medians:", median_from, median_to, median_salary)

# Imputing missing salaries, but not experience
df = df.fillna({
    "SALARY_FROM": median_from,
    "SALARY_TO": median_to,
    "SALARY": median_salary
})

# Computing average salary
df = df.withColumn(
    "Average_Salary", (col("SALARY_FROM") + col("SALARY_TO")) / 2
)

# Selecting required columns
export_cols = [
    "Average_Salary",
    "SALARY",
    "EDUCATION_LEVELS_NAME",
    "REMOTE_TYPE_NAME",
    "MAX_YEARS_EXPERIENCE",
    "LOT_V6_SPECIALIZED_OCCUPATION_NAME"
]

df_selected = df.select(*export_cols)

pdf_selected = df_selected.toPandas()
pdf_selected.head()

## Cleaning Education column and  Exporting Cleaned Data

*I referred to Claude Sonnet 4 for prompts and sample code ideas, but I wrote and adapted the final implementation myself

In [ ]:
# To remove \n and \r
pdf_selected["EDUCATION_LEVELS_NAME"] = (
    pdf_selected["EDUCATION_LEVELS_NAME"]
    .astype(str)               
    .str.replace(r"[\n\r]", "", regex=True)
    .str.strip()                
)
pdf_selected.to_csv("data/lightcast_cleaned.csv", index=False)
pdf_selected.head()

## Exporting Cleaned Data

In [ ]:
print("Data cleaning complete. Rows retained:", len(pdf_selected))

# Query 1: Salary Distribution by Industry and Employment Type

## Salary Distribution by Employment Type

In [ ]:
import re
import plotly.io as pio

df_query1 = df.select(
    "EMPLOYMENT_TYPE_NAME",
    "NAICS2_NAME",
    "SALARY"
)

pdf_query1 = df_query1.toPandas()

pdf_query1 = pdf_query1[pdf_query1["SALARY"] > 0]

pdf_query1["EMPLOYMENT_TYPE_NAME"] = (
    pdf_query1["EMPLOYMENT_TYPE_NAME"]
    .astype(str) 
    .apply(lambda x: re.sub(r"[^\x00-\x7F]+", "", x)) 
    .str.strip()  
)
pdf_query1 = pdf_query1[pdf_query1["EMPLOYMENT_TYPE_NAME"] != "None"]

median_salaries = pdf_query1.groupby("EMPLOYMENT_TYPE_NAME")["SALARY"].median()

sorted_employment_types = median_salaries.sort_values(ascending=False).index

pdf_query1["EMPLOYMENT_TYPE_NAME"] = pd.Categorical(
    pdf_query1["EMPLOYMENT_TYPE_NAME"],
    categories=sorted_employment_types,
    ordered=True
)


In [ ]:
fig = px.box(
    pdf_query1,
    x="EMPLOYMENT_TYPE_NAME",
    y="SALARY",
    title="Salary Distribution by Employment Type",
    color_discrete_sequence=["#CC0000"],  
    boxmode="group",
    points="outliers" 
)


fig.update_layout(
    title=dict(
        text="Salary Distribution by Employment Type",
        font=dict(size=20, family="Arial Black", color="black"),  
        x=0.5  
    ),
    xaxis=dict(
        title=dict(
            text="Employment Type",
            font=dict(size=14, family="Arial Black", color="black")  
        ),
        tickangle=-45,
        tickfont=dict(size=12, family="Arial Black", color="black"),  
        showline=True,
        linecolor="black",
        linewidth=2,
        mirror=True,
        showgrid=False,  
        categoryorder="array",
        categoryarray=sorted_employment_types.tolist() 
    ),
    yaxis=dict(
        title=dict(
            text="Salary (1000 $)",
            font=dict(size=14, family="Arial Black", color="black")  
        ),
        tickvals=[0, 50000, 100000, 150000, 200000, 250000, 300000, 350000, 400000, 450000, 500000],
        ticktext=["0", "50k", "100k", "150k", "200k", "250k", "300k", "350k", "400k", "450k", "500k"],
        tickfont=dict(size=12, family="Arial Black", color="black"), 
        showline=True,
        linecolor="black",
        linewidth=2,
        mirror=True,
        showgrid=True,          
        gridcolor="lightgray", 
        gridwidth=0.5             
    ),
    font=dict(family="Arial", size=16, color="black"),
    boxgap=0.7,
    plot_bgcolor="white",
    paper_bgcolor="white",
    showlegend=False,
    height=500,
    width=850,
)

# Show the figure
fig.show()
fig.write_html("output/Q1.html")
fig.write_image("output/Q1.png", width = 850, height = 500, scale=1)


![](./output/Q1.png){ width=90% }